In [15]:
import torch

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [17]:
df = pd.read_csv('Churn_Modelling.csv')

In [18]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [19]:
X = df.loc[:, 'CreditScore':'EstimatedSalary']
y = df['Exited']

In [20]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(drop='first', sparse_output=False, dtype=int)
encoded = ohe.fit_transform(X[['Geography', 'Gender']])

X = X.drop(columns=['Geography', 'Gender'])
X = pd.concat([X, pd.DataFrame(encoded, columns=ohe.get_feature_names_out())], axis=1)

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
print('Shape of X_train:',X_train.shape)
print('Shape of y_train:',y_train.shape)
print('Shape of X_test:',X_test.shape)
print('Shape of y_test:',y_test.shape)

Shape of X_train: (8000, 11)
Shape of y_train: (8000,)
Shape of X_test: (2000, 11)
Shape of y_test: (2000,)


In [34]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [37]:
from torch.utils.data import DataLoader, TensorDataset

# Assuming X_train and y_train are NumPy arrays, convert them to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)  # Reshape for binary classification

# Create training and validation datasets (33% validation split)
train_size = int(0.67 * len(X_train))
val_size = len(X_train) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(
    TensorDataset(X_train_tensor, y_train_tensor), [train_size, val_size]
)

# Create DataLoaders
batch_size = 10
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(11, 11)  # Input layer
        self.fc2 = nn.Linear(11, 7)   # First hidden layer
        self.dropout = nn.Dropout(0.2)  # Dropout layer
        self.fc3 = nn.Linear(7, 6)    # Second hidden layer
        self.fc4 = nn.Linear(6, 1)    # Output layer

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))
        return x

# Initialize the model
classifier = Classifier()

In [39]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(11, 11)
        self.fc2 = nn.Linear(11, 7)
        self.dropout = nn.Dropout(0.2)
        self.fc3 = nn.Linear(7, 6)
        self.fc4 = nn.Linear(6, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))
        return x

classifier = Classifier()

In [40]:
import torch.optim as optim
from torch import nn

optimizer = optim.Adam(classifier.parameters(), lr=0.01)
criterion = nn.BCELoss()  # Binary Cross Entropy Loss

In [42]:
epochs = 5
patience = 20
min_delta = 0.0001
best_val_loss = np.inf
epochs_no_improve = 0

for epoch in range(epochs):
    classifier.train()
    train_loss = 0.0
    
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = classifier(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    train_loss = train_loss/len(train_loader)

    classifier.eval()
    val_loss = 0.0
    for batch_X, batch_y in val_loader:
        outputs = classifier(batch_X)
        loss = criterion(outputs, batch_y)
        val_loss += loss.item()

    val_loss /= len(val_loader)

    # Early Stopping
    if val_loss < best_val_loss - min_delta:
        best_val_loss = val_loss
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1

    if epochs_no_improve >= patience:
        print("Early stopping triggered.")
        break

In [45]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score

# Convert X_test DataFrame to PyTorch tensor
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

# Set model to evaluation mode
classifier.eval()

# Make predictions
with torch.no_grad():
    y_pred = classifier(X_test_tensor)

# Convert probabilities to binary values (threshold at 0.5)
y_pred = (y_pred.numpy() >= 0.5).astype(int)

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)

# Compute accuracy
score = accuracy_score(y_test, y_pred)
print("The accuracy of the model on the test dataset is", score)

Confusion Matrix:
 [[1534   73]
 [ 206  187]]
The accuracy of the model on the test dataset is 0.8605
